# Icing Detection for Anemometers

<p>
    <img src='../images/anemometer_device.png' width=200>
    <p>
        <center>
            <strong>Figure:</strong>Example of an anemometer
        </center>
    </p>
</p>

<p>
    The figure above shows an anemometer that is typically used to measure wind speed. Essentially, the cups are being hit by the wind, so that the top part is spinning. The speed of the mechanical rotation is then used as to calculate the wind speed.
</p>

<p>
    As the top part must physically move in order to measure the wind speed, freezing temperatures can be detrimental for such a task. The degree to which an anemometer is frozen can be classified into two classes: partially, and fully. Below figure depicts how each class can look like.
</p>

<p>
    <img src='../images/anemometer_icing.jpg' width=500>
    <p>
        <center>
            <strong>Figure:</strong>Example of a fully iced anemometer (left) and a partially iced anemometer (right)
        </center>
    </p>
</p>

<p>
    A fully iced anemometer will not be able to rotate anymore. A partially iced anemometer will still rotate. However, due to a higher degree of friction caused by the icing, it will not rotate as fast as the wind speed hitting it. Hence, while the actual wind speed might be 10 m/s, the sensor will only read 5 m/s.
</p>

<p>
    The plot below shows the temperature on the top, and the wind speed of two anemometers on the same height. All three sensors are mounted to the same met mast. While the temperature is above 0°C, both wind speed sensors are reading almost the same wind speed. This is to be expected as both sensors are on the same height of the met mast with a horizontal distance of around 4 metres.
</p>
<p>
    On February 20th, Anemometer B slowly ices. At first, the sensor is only partially iced. It still measures a wind speed above 0 m/s, but in comparison to Anemometer A it is much lower. On February 21st, Anemometer B is fully iced.
</p>

<p>
    <img width="900" height="500" src="../images/icing_timeseries_example.png">
    <a href="../plots/icing_timeseries_example.html">Download Plotly File</a>
</p>

<p>
    If two anemometers are mounted on the same height, both of their wind speed measurements are to be expected almost equal. The figure below shows both anemometers plotted against each other. The red line represents equality between both sensors. The plot on the left side shows all data points that occured during a time in which icing could have been possible (below 0°C within the last two days) and on the right vice versa.  
</p>
<p>
    In the right plot, you can also see a few points that are deviating from the red line. These points are caused by <i>tower shadowing</i>.
</p>

<p>
    <img width="900" height="500" src="../images/icing_scatter_60m_raw.png">
    <a href="../plots/icing_scatter_60m_raw.html">Download Plotly File</a>
</p>

<p>
    <i>Tower shadowing</i> is an alteration of the uniform wind flow caused by the presence of the tower. The figure on the left side shows how a met mast usually looks like. Anemometers are mounted on the same height 180° from each other. The right side of the figure shows how the wind flow gets affected by the <i>tower shadow effect</i>. If the wind direction in the example below comes directly from the left, the anemometer on the left side will measure the wind speed correctly. However, the anemometer on the right side will be in an area affected by the wind shadow. Hence, the anemometer on the right side will have lower readings than the left side.
</p>

<p>
    <img src='../images/anemometer_shadow_tower.png' width=800>
    <p>
        <center>
            <strong>Figure:</strong> Wind field predicted for 10 m/s ambient wind. (<a href='http://www.iawe.org/Proceedings/11ACWE/11ACWE-Lubitz.pdf'>Source</a>)
        </center>
    </p>
</p>

<p>
    Partial icing and the <i>tower shadow effect</i> could look very similar in a time series. In both instances, one anemometer will be reading lower than its counterpart on the same height. In order to account for the latter, it is necessarily to calculate the shadow sectors for each anemometer pair. Data points that are affected by it will be processed at a later point to correct the wind flow.
</p>
<p>
    The figure below shows the tower shadow plot with the wind direction on the x-axis, and the difference between both anemometers on the y-axis. The median delta at each degree is presented in red. Dark blue marks the shadow sectors.
</p>

<p>
    <img width="900" height="500" src="../images/icing_towershadow_scatter.png">
    <a href="../plots/icing_towershadow_scatter.html">Download Plotly File</a>
</p>

<p>
    The algorithm is pulling the position of each anemometer from a master database. Each sensor's tower shadow will be centered directly on the opposing side. The sector is not expected to be larger than 50° to each side. Hence, the algorithm is only looking at a 100° large section. Change point analysis is then being conducted on this section to find the two points in which the shadow section starts and ends.
</p>

In [1]:
import numpy as np
import pandas as pd

def _group_DIR(data, pair):
    """The algorithm finds the closest value in an array to a specified value.
    
    Args:
        data (DataFrame): DataFrame that includes two wind speed sensor (on a 
                            similiar height) and one wind direction sensor 
                            measurements.
        pair (list): List that includes two wind speed and one wind direction 
                            sensors

    Returns:
        data_ratio (DataFrame): Median delta between both wind speed sensors as 
                            function of wind direction
    """
    
    data = data.copy()
    
    # Filtering out low wind speeds
    data = data.loc[data.iloc[:, 0] > 4]
    data = data.loc[data.iloc[:, 1] > 4]
    
    # Calculating median ratio between both wind speed sensor as function of 
    # wind direction
    data['ratio'] = data.iloc[:, 0]-data.iloc[:, 1]
    data_grouped = data.groupby(pair[2]).median()
    
    data_ratio = data_grouped[['ratio']]
    
    return data_ratio

def _find_closest_value(value, X): 
    """The algorithm finds the closest value in an array to a specified value.
    
    Args:
        value (float): Specified value.
        X (array): Array to search for closest value in.

    Returns:
        value_closest (float): Closest value of array.
    """
    
    value_closest = X[np.unravel_index(np.argmin(np.abs(X-value)), X.shape)]
    
    return value_closest

def _find_direction_breakpoints(data, number_bkpts):
    """Detects structural change points in a data set.

    Args:
        data (DataFrame): Data that will be checked for change points.
        number_bkpts (int): Number of expected change points.

    Returns:
        bkpts (list): List with indices of detected change points.
    """

    model = 'l2'
    algo = rpt.Dynp(model=model, min_size=3, jump=5).fit(np.array(data))        
    bkpts = algo.predict(n_bkps=number_bkpts)
    
    return bkpts

def _detect_shadow_range(data, pairs, position, number_bkpts):
    """Detects shadow sectors for wind speed sensor.

    Args:
        data (DataFrame): Data that will be checked for change points.
        pairs (list): Wind speed and direction sensors pair.
        position (float): Position of to be checked wind speed sensor.
        number_bkpts (int): Number of expected change points.

    Returns:
        shadow_range (list): Shadow sector interval of wind speed sensor.
    """    
    
    # Calculate ratio of wind speed sensor pair as function of wind direction
    cum_pair = _group_DIR(data, pairs)
    cum_pair = cum_pair.interpolate('linear')

    cum_pair_add = cum_pair.copy()
    cum_pair_sub = cum_pair.copy()

    cum_pair_add.index += 360
    cum_pair_sub.index -= 360

    cum_pair = pd.concat([cum_pair_sub, cum_pair, cum_pair_add], axis=0)
      
    # Detects left and right side of shadow sector
    position = position + 180 if position + 180 < 360 else position - 180

    a = _find_closest_value(position-50, cum_pair.index)
    b = _find_closest_value(position+50, cum_pair.index)

    cum_pair = cum_pair.loc[a:b]

    bkpts = np.array([cum_pair.index[i] for i in _find_direction_breakpoints(\
                                                cum_pair, number_bkpts)[:-1]])

    shadow_range = [bkpts[0]-15 if bkpts[0]-15>0 else bkpts[0]-15+360,
                    bkpts[1]+15 if bkpts[1]+15<360 else bkpts[1]+15-360]

    return shadow_range

<p>
    After the shadow sectors have been flagged and removed for further analysis, the icing detection begins. The algorithm is only looking at data points that have been affected by below 0°C temperatures in the last two days.
</p>
<p>
    In a first step, the algorithm is checking for fully iced data points. Anemometer by anemometer the script is flagging flat lines. Flat lines are consecutive points with the same value. Or in other words for consecutive time stamps the value of each point is not deviating from each other.
</p>
<p>
    In a second step, the algorithm is checking for partially iced data points. The script is putting each anemometer in relation to all the other anemometers that are mounted to the same met mast. Below figure shows that how two anemometers on the same height (left) will have very similar readings, while two anemometers on different height (right) will have more spread out readings. The spread of the difference between both sensors will change with wind speed. Low and high wind speeds will experience less spread, medium speeds more. The result is an oval-like shape. The algorithm is taking this into account. Per 1 m/s of wind speed the script is calculating the median delta and the median absolute deviation.
</p>

<p>
    <img width="900" height="500" src="../images/icing_scatter_height.png">
    <a href="../plots/icing_scatter_height.html">Download Plotly File</a>
</p>

<p>
    In a second step, the algorithm is checking for partially iced data points. The script is putting each anemometer in relation to all the other anemometers that are mounted to the same met mast. The anemometer on the x-axis is the sensor that will be checked, and the one on the y-axis will be the reference sensor. The idea is essentially to check if their is too large of a difference between both sensors.
</p>
<p>
    Above figure shows  how two anemometers on the same height (left) will have very similar readings, while two anemometers on different height (right) will have more spread between their readings. The spread of the difference between both sensors will change with wind speed. Low and high wind speeds will experience less spread, medium speeds more. The result is an oval-like shape. This is being taken into account by the algorithm. The script is binning the anemometer on the x-axis per 1 m/s and calculating the median delta and median absolute deviation between the tested and the reference sensor per 1 m/s. If the tested sensor is more than then median plus 4 times the median absolute deviation smaller than the reference, it will be flagged as partial icing.
</p>
<p>
    Below figure shows the result of the icing detection after fully and partially iced data points have been flagged and removed. It seems to have been quite successful at detecting the previous flagged interval from February 20th to February 23rd. However, the plot is also missing the previously flagged shadow sectors.
</p>

<p>
    <img width="900" height="500" src="../images/icing_timeseries_noshadow_flagged.png">
    <a href="../plots/icing_timeseries_noshadow_flagged.html">Download Plotly File</a>
</p>

In [2]:
def _detect_outlier_icing_full(data, channel, channel_temp):
    """Detects fully iced wind speed sensors.

    Args:
        data (DataFrame): Data that will be checked for full icing.
        channel (str): Wind speed sensor that will be checked for icing.
        channel_temp (str): Temperature sensor of that mast.

    Returns:
        data_index (DataFrame): DataFrame with all fully iced data points.
    """
    
    data = data[[channel, channel_temp]].copy()
    
    # Selects only relevant time intervals
    data_frozen = data[channel_temp].rolling(24*6*1).min().shift(1) <= 0
    data = data.loc[data_frozen]
    
    # Flags fully iced data points
    data.loc[(data[channel].rolling(6).std().fillna(method='bfill') < 0.05), 
             'OUT'] = True
    
    for i in range(1, 6):
        data.OUT += data.OUT.shift(-1).fillna(0)
        data.OUT += data.OUT.shift(1).fillna(0)
    
    data.loc[data.OUT >= 1, 'OUT'] = 1
    
    data_index = data[data.OUT == 1]
    
def _detect_outlier_icing_partial(data, channel, channel_temp):
    """Detects partially iced wind speed sensors.

    Args:
        data (DataFrame): Data that will be checked for partial icing.
        channel (str): Wind speed sensor that will be checked for icing.
        channel_temp (str): Temperature sensor of that mast.

    Returns:
        data_index (DataFrame): DataFrame with all partially iced data points.
    """
    
    data_channel = data[[channel, channel_temp]].copy()
    
    # Selects all available wind speed sensors in mast
    cols = [i for i in data.columns if 'SPD' in i and i!=channel]

    for col in cols:

        data_merged = pd.concat([data_channel,
                                 data[[col]].copy()], axis=1)
        
        # Selects only relevant time intervals
        data_frozen = data_merged[channel_temp].rolling(24*6*2).min().shift(1) <= 0 
        data_merged = data_merged.loc[data_frozen]
        
        # Assignes relevant bins to corresponding data points
        data_merged['buckets'] = pd.cut(data_merged[col], bins=
                                        round(data_merged[col].max()))
        data_merged['delta'] = data_merged[channel] - data_merged[col]
        
        # Calculates each bins' allowed delta threshold
        thresh_dict = {}
        for bucket in data_merged.buckets.unique().dropna():
            
            thresh_dict[bucket] = data_merged.groupby('buckets').delta.median()[bucket] - \
                                    4*data_merged.groupby('buckets').delta.mad()[bucket]        

        data_merged['thresh'] = data_merged.buckets.apply(lambda row: thresh_dict[row])
        data_merged.thresh = data_merged.thresh.astype('float')
    
        # Flags partially iced data points
        data_merged.loc[data_merged.delta < data_merged.thresh, 'OUT'] = True
        
        data_channel.loc[data_merged[data_merged.OUT == True].index, 'OUT'] = True

    data_index = data_channel[data_channel.OUT == True]
    
    return data_index

<p>
    The shadow sectors were flagged previously to flag icing with greater accuracy. However, icing is not exclusive to certain wind directions. That means, icing can also occur within shadow sectors. Before the wind flow can be corrected for the shadow at a later point, icing still needs to be flagged for those parts of well.
</p>
<p>
    Hence, in a third step, the shadow sectors are being imputed again. The algorithm for fully iced data points is being leveraged again. Additionally, all previously flagged data points for icing are being taken into account. If an anemometer was flagged, the date from that timestamp is extracted. Any data point of that anemometer on that date (+/- one additional date) that is also in a shadow sector is flagged as icing. That way partial icing is flagged for the shadow sectors as well.
</p>
<p>
    Below figure shows the time series without any iced data points and including the shadow sectors. The previously marked interval from February 20th to February 23rd was properly flagged as icing. Additionally, we can be sure that shadow sectors do not have any iced data points.
</p>

<p>
    <img width="900" height="500" src="../images/icing_timeseries_flagged.png">
    <a href="../plots/icing_timeseries_flagged.html">Download Plotly File</a>
</p>

In [3]:
def _impute_shadow_sectors(data_clean, data_raw, shadow1, shadow2, spd, spd_dir):
    """Detects partially iced wind speed sensors.

    Args:
        data_clean (DataFrame): Cleaned sensor data.
        data_raw (DataFrame): Raw sensor data.
        shadow1 (float): Left side of shadow sector interval.
        shadow2 (float): Right side of shadow sector interval.
        spd (str): Wind speed sensor to be imputed.
        spd_dir (str): Wind direction sensor.
    """
    
    data_clean.loc[(data_raw[spd_dir] > shadow1)&(data_raw[spd_dir] < shadow2), spd] = \
                        data_raw.loc[(data_raw[spd_dir] > shadow1)&
                        (data_raw[spd_dir] < shadow2), spd]
    
def _detect_outlier_icing_shadow(data, col_out, spd_dir, shadow1, shadow2):
    """Detects icing in shadow sectors of wind speed sensor.

    Args:
        data (DataFrame): Data whose shadow sectors will be checked for icing.
        col_out (DataFrame): Data points of non shadow sectors that is flagged for icing.
        spd_dir (str): Wind direction sensor.
        shadow1 (float): Left side of shadow sector interval.
        shadow2 (float): Right side of shadow sector interval.
        
    Returns:
        data_index (DataFrame): DataFrame with all iced data points in shadow sector.
    """
    
    icing_index_daily = data.loc[data[col_out]==True].index.to_period('D')
    icing_range = [pd.date_range(i.start_time, i.end_time, freq='10MIN') for i in \
                   icing_index_daily]
    
    icing_index = []
    for i in icing_range:
        icing_index.extend(i)
        
    icing_index = np.array(icing_index)
                
    shadow_index = data.loc[(data[spd_dir] > shadow1)&(data[spd_dir] < shadow2)].index    
        
    icing_index_base = icing_index.copy()
        
    for i in range(1, 2):
                        
        icing_plus = icing_index_base + pd.Timedelta(i,'D')
        icing_minus = icing_index_base - pd.Timedelta(i,'D')

        icing_index = np.append(icing_index, icing_plus)
        icing_index = np.append(icing_index, icing_minus)
                
    icing_index = set(icing_index) 
    shadow_index = set(shadow_index)
    
    data_index = icing_index.intersection(shadow_index)
    
    return data_index